In [ ]:
import pandas as pd
from connect import PostgreSQL
from tqdm import tqdm
import datetime as dt
from datetime import date
import plotly.graph_objs as go

# Parâmetros

In [ ]:
_schema = 'D-1'

In [ ]:
_pixels = 720

pixels_width = int((_pixels/9)*16)
pixels_height = int(_pixels)

In [ ]:
date_query = date(int((dt.datetime.now().strftime('%Y-%m-%d').split('-')[0])), int((dt.datetime.now().strftime('%Y-%m-%d').split('-')[1])), 1).strftime('%Y-%m-%d')

In [ ]:
codigos_kaizen = [('KAIZEN ASA NORTE', '13996'),
                  ('KAIZEN CEILÂNDIA', '16100'),
                  (     'KAIZEN GAMA', '18400'),
                  (  'KAIZEN SOF SUL', '20000')]

In [ ]:
codigos_vended = [(    'GARANTIA', '0100'),
                  ('MARIO SERGIO', '0001'),
                  (   'DEVOLUCAO', '0006')]

In [ ]:
codigos_cfop = [('1949'),
                ('2949'),
                ('1603')]

In [ ]:
lista_lojas = (['01',             'CD'],
               ['02',               ''],
               ['03',  '04. ASA NORTE'],
               ['04',  '02. CEILÂNDIA'],
               ['05',       '03. GAMA'],
               ['06',    '01. SOF SUL'],
               ['VO', '05. E-COMMERCE'])

In [ ]:
relatorio = 'ACOMPANHAMENTO DE VENDA'

relatorio_lower = relatorio.lower()
relatorio_msg = relatorio_lower.replace(relatorio_lower.split()[0], relatorio_lower.split()[0].title())

In [ ]:
caminho_imagem = 'Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\REPORT'

In [ ]:
contatos = [('PrimeiraFoto', 'https://chat.whatsapp.com/LK4LDajkH52BNydoLGGD2d', True),
            ('Dados e Int',  'https://chat.whatsapp.com/JxY14kWarG70MxB2jzb5pc', False),
            ('Dream Team',   'https://chat.whatsapp.com/Hb0KMgYnuFB8coTPlFKTzJ', False),
            ('Report Di',    'https://chat.whatsapp.com/G4p8qdUIUblCNPS80244RQ', False),
            ('API',          'https://chat.whatsapp.com/EBLX0fxYfvuHL7jZ9CZECQ', True),
            ('Teste Python', 'https://chat.whatsapp.com/I4y7EAVfns629NvfoLUtGI', True)]

#

# DataFrames

## Clientes

In [ ]:
df_cliente = PostgreSQL.read_postgres(name_table='cliente', schema=_schema)

In [ ]:
df_cli_ped = PostgreSQL.read_postgres(name_table='cli_ped', schema=_schema, query=f'WHERE DT_CADASTR >= \'{date_query}\'')

## Pedidos

In [ ]:
df_pedido = PostgreSQL.read_postgres(name_table='pedido', schema=_schema, query=f'WHERE DT_EMISSAO >= \'{date_query}\'')

In [ ]:
df_pedido['VALOR_TOT'] = df_pedido['VALOR_TOT'].astype(float)

In [ ]:
for (kaizen, cod_kaizen) in tqdm(codigos_kaizen):
    df_pedido = df_pedido[df_pedido['CODCLI'] != cod_kaizen]

In [ ]:
for (vendedor, cod_vende) in tqdm(codigos_kaizen):
    df_pedido = df_pedido[df_pedido['CODCLI'] != cod_vende]

In [ ]:
df_pedido = df_pedido[df_pedido['TIPPED'] == 'V']

In [ ]:
df_pedido = df_pedido[(df_pedido['OBSERVA'].str.contains('REQ') != True) |
                      (df_pedido['OBSERVA2'].str.contains('REQ') != True)]

In [ ]:
for i in tqdm(df_pedido.index):
    if df_pedido.loc[i, 'FORMA_PGTO'] == 'M':
        df_pedido.loc[i, 'COD_LOJA'] = 'VO'
    else:
        df_pedido.loc[i, 'COD_LOJA'] = df_pedido.loc[i, 'CD_LOJA']

In [ ]:
for i in tqdm(df_pedido.index):
    if df_pedido.loc[i, 'COD_LOJA'] == 'VO':
        df_pedido.loc[i, 'NU_CPFCNPJ'] = list(df_cli_ped[(df_cli_ped['CD_LOJA'] == df_pedido.loc[i, 'CD_LOJA']) &
                                             (df_cli_ped['SG_PEDIDO'] == df_pedido.loc[i, 'SERIE']) &
                                             (df_cli_ped['NU_PEDIDO'] == df_pedido.loc[i, 'NU_NOTA'])]['NU_CPFCNPJ'])[0]

In [ ]:
df_pedido = pd.merge(df_pedido, df_cliente, how='inner', on='CODCLI')

In [ ]:
for i in tqdm(df_pedido.index):

    if df_pedido.loc[i, 'COD_LOJA'] == 'VO':
        identify = df_pedido.loc[i, 'NU_CPFCNPJ']
    else:
        identify = df_pedido.loc[i, 'CPF_CGC']

    if df_pedido.loc[i, 'CODAREA'] == '104':
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CNPJ*'
    elif len(identify) < 14:
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CPF'
    else:
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CNPJ'

## Devolução

In [ ]:
df_prod_ent = PostgreSQL.read_postgres(full_query=f'select a.*, b.IN_CLIFOR as IN_CLIFOR1, b.IN_CANCELA, b.NFEENVSTAT, c.FORMA_PGTO from "{_schema}".prod_ent a inner join "{_schema}".entrada b on a.CD_LOJA = b.CD_LOJA and a.SG_SERIE = b.SG_SERIE and a.NU_NOTA = b.NU_NOTA inner join "{_schema}".venda c on a.CD_LOJA = c.CD_LOJA and a.SG_ORIGEM = c.SERIE and a.NU_ORIGEM = c.NU_NOTA where a.DT_EMISSAO >= \'{date_query}\'')

In [ ]:
for (kaizen, cod_kaizen) in tqdm(codigos_kaizen):
    df_prod_ent = df_prod_ent[df_prod_ent['CD_CLIENTE'] != cod_kaizen]

In [ ]:
for (cod_cfop) in tqdm(codigos_cfop):
    df_prod_ent = df_prod_ent[df_prod_ent['CD_CFOP'] != cod_cfop]

In [ ]:
df_prod_ent = df_prod_ent[df_prod_ent['IN_CLIFOR1'] == 'C']

In [ ]:
for nfestatus in tqdm(df_prod_ent['NFEENVSTAT']):
    if nfestatus != None:
        if 'DENEG' in nfestatus.upper():
            df_prod_ent = df_prod_ent[df_prod_ent['NFEENVSTAT'] != nfestatus]

In [ ]:
for i in tqdm(df_prod_ent.index):
    if df_prod_ent.loc[i, 'FORMA_PGTO'] == 'M':
        df_prod_ent.loc[i, 'COD_LOJA'] = 'VO'
    else:
        df_prod_ent.loc[i, 'COD_LOJA'] = df_prod_ent.loc[i, 'CD_LOJA']

    df_prod_ent.loc[i, 'VL_TOTAL_DEV'] = float(df_prod_ent.loc[i, 'VL_PRECO']) * float(df_prod_ent.loc[i, 'QT_DEVOLVE'])

In [ ]:
df_prod_ent = df_prod_ent.rename(columns={'CD_CLIENTE':'CODCLI'})
df_prod_ent = pd.merge(df_prod_ent, df_cliente, how='inner', on='CODCLI')

In [ ]:
for i in tqdm(df_prod_ent.index):

    identify = df_prod_ent.loc[i, 'CPF_CGC']

    if df_prod_ent.loc[i, 'CODAREA'] == '104':
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CNPJ*'
    elif len(identify) < 14:
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CPF'
    else:
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CNPJ'

## Metas

In [ ]:
df_metas = pd.read_excel('Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\PROJETOS\\EXCEL\\BASES\\DATAS.xlsx', sheet_name='Metas')

In [ ]:
df_data = pd.read_excel('Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\PROJETOS\\EXCEL\\BASES\\DATAS.xlsx', sheet_name='Datas')

In [ ]:
df_datas = df_data[df_data['Mês - Ano'] == df_data[df_data['Data'] == date_query]['Mês - Ano'].values[0]].reset_index(drop=True)

In [ ]:
df_meta_mes = df_metas[df_metas['Mês - Ano'] == df_datas['Mês - Ano'][0]]

In [ ]:
df_meta_data = df_datas.drop(columns=['Dia', 'Mês', 'Ano', 'Dia da Semana', 'Nome - Mês', 'Semestre', 'Trimestre', 'Sequencial - Dia da Semana', 'Dias Úteis']).merge(df_meta_mes.drop(columns=['Mês', 'Ano', 'Dias Úteis', 'Qtd Sábados', 'Nome - Mês', 'Loja']), how='inner', on='Mês - Ano')

In [ ]:
for i in df_meta_data.index:
    if df_meta_data.loc[i, 'Dia Útil'] == 'Sim':
        df_meta_data.loc[i, 'Meta'] = df_meta_data.loc[i, 'Meta Diária']
    
    elif df_meta_data.loc[i, 'Dia Útil'] == 'Meio':
        df_meta_data.loc[i, 'Meta'] = df_meta_data.loc[i, 'Meta Sábado']

    else:
        df_meta_data.loc[i, 'Meta'] = 0

df_meta_data = df_meta_data.rename(columns={'Cd_Loja':'CD_LOJA', 'Data':'DT_EMISSAO'})

# Cálculo

In [ ]:
df_venda_bruta = df_pedido.groupby(['COD_LOJA', 'DT_EMISSAO', 'TIPO_CLIENTE']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

In [ ]:
df_devolucao = df_prod_ent.groupby(['COD_LOJA', 'DT_EMISSAO', 'TIPO_CLIENTE']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

In [ ]:
df_pedido[df_pedido['TIPO_CLIENTE'] != 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

In [ ]:
df_venda_bruta_cnpj = df_venda_bruta

In [ ]:
df_final = pd.merge(df_venda_bruta, df_devolucao, how='outer', on=['CD_LOJA', 'DT_EMISSAO'])

df_final = df_final.sort_values(by=['DT_EMISSAO', 'CD_LOJA'])
df_final = df_final.reset_index(drop=True)

df_final = pd.merge(df_meta_data, df_final, how='left', on=['CD_LOJA', 'DT_EMISSAO'])

In [ ]:
df_final = df_final.fillna(0)

In [ ]:
df_hoje = df_final
df_hoje = df_hoje.rename(columns={'VALOR_TOT':'VENDA BRUTA', 'VL_TOTAL_DEV':'DEVOLUÇÃO', 'Meta':'META'})

for i in df_hoje.index:
    df_hoje.loc[i, 'VENDA LÍQUIDA'] = (df_hoje.loc[i, 'VENDA BRUTA'] - df_hoje.loc[i, 'DEVOLUÇÃO'])
    df_hoje.loc[i, '% DEV'] = df_hoje.loc[i, 'DEVOLUÇÃO'] / df_hoje.loc[i, 'VENDA BRUTA']
    
    client_cgc = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                     (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                     (df_pedido['TIPO_CLIENTE'] != 'CPF')]['CPF_CGC'].drop_duplicates())

    client_cpf = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                     (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                     (df_pedido['TIPO_CLIENTE'] == 'CPF')]['CPF_CGC'])

    venda_cgc = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                     (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                     (df_pedido['TIPO_CLIENTE'] != 'CPF')]['VALOR_TOT'])

    venda_cpf = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                     (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                     (df_pedido['TIPO_CLIENTE'] == 'CPF')]['VALOR_TOT'])

    df_hoje.loc[i, 'VENDA_CNPJ'] = sum(venda_cgc)
    df_hoje.loc[i, 'VENDA_CPF'] = sum(venda_cpf)

    df_hoje.loc[i, 'CNPJ'] = len(client_cgc)
    df_hoje.loc[i, 'CPF'] = len(client_cpf)

In [ ]:
df_hoje

In [ ]:
df = df_hoje[df_hoje['CD_LOJA'] == '05']
df[['DT_EMISSAO', 'Nome - Dia da Semana', 'META', 'VENDA BRUTA', 'VENDA LÍQUIDA', 'DEVOLUÇÃO', '% DEV', 'VENDA_CNPJ', 'CNPJ', 'VENDA_CPF', 'CPF']]